In [ ]:
import sys
sys.path.append('../mypy/MyModel')
import os
import json
import datetime
import pandas as pd
import numpy as np
import days_calculate

class Customs:
    def __init__(self):
        with open('fittingRecord.config','r',encoding='utf-8') as f:
            lines=f.readlines()
        _line=''
        for line in lines:
            newLine=line.strip('\n')
            _line=_line+newLine
        config=json.loads(_line)
        
        self.default_dir=config['基本文件夹']
        
        ItemOxy=pd.read_excel(os.path.join(self.default_dir,'训练项目.xlsx'),sheet_name=0,skiprows=0)
        self.ItemOxy=ItemOxy['名称'].tolist()        
        
    def read_xlsx(self,cus_name):
        cus_dir=os.path.join(self.default_dir,'客户',cus_name)
        cus_xlsx=os.path.join(cus_dir,'基本情况及训练记录-'+cus_name+'.xlsx')
        
        cus_basic=pd.read_excel(cus_xlsx,sheet_name=0,nrows=1)
        cus_basic.dropna(axis=1,how="any",inplace=True)
        cus_basic.columns=['姓名','性别','出生日期']
        
        birthDay=str(cus_basic['出生日期'].tolist()[0])[0:10].replace('-','')
        #年龄
        age=days_calculate.calculate_age(birthDay)
        
        cus_body=pd.read_excel(cus_xlsx,sheet_name=0,header=None,skiprows=5,names=['日期','身高','体重','BMI','体脂率','胸围','臂围','腰围','臀围','大腿围'])
        cus_body.dropna(axis=0,how="any",inplace=True)
        
        cus_train=pd.read_excel(cus_xlsx,sheet_name=1,skiprows=1,names=['日期','时间','项目','第1组重量','第1组次数','第2组重量','第2组次数','第3组重量','第3组次数','第4组重量','第4组次数','第5组重量','第5组次数','时长','消耗热量','训练过程'])
                
        #总的上课次数（统计去重之后的日期）
        totalTimes=cus_train['日期'].nunique() 
        
        #第一次上课到今天的日期（统计去重之后的日期）
        FirstTrainDate=cus_train['日期'].min().strftime('%Y%m%d')
        trainDays=days_calculate.calculate_days(FirstTrainDate)

        
        out=Vividict()
        
        #姓名、性别
        out['basicInfo']['name']=cus_name
        out['basicInfo']['gender']=cus_basic['性别'].tolist()[0]   
        out['basicInfo']['age']=age
        
        out['training']['totalTimes']=totalTimes
        
        #身体数据
        body=np.array(cus_body.iloc[:,1:]).tolist()
        for n,dt in enumerate(cus_body['日期']):
            MsrDate=str(dt)[0:10].replace('-','')
            out['body'][MsrDate]=body[n]
        
        
        #按日期计算训练项目细节
        for dt in cus_train['日期'].unique():        
            trainTime=str(dt)[0:10].replace('-','')
            today_train=cus_train.loc[cus_train['日期']==trainTime]
            trainList=today_train['项目'].tolist()
            trainOxyToday=[]
            trainNonOxyToday=[]

            #训练详细情况（训练项目分类、次数等）
            for trainItem in trainList: 
                if trainItem not in self.ItemOxy: #无氧
                    _trainTimes=[]
                    for k in range(5):               
                        _trainTimes.append(today_train[today_train['项目']==trainItem]['第'+str(k+1)+'组次数'].tolist()[0])
                    train_counts=np.array(_trainTimes)
                    trainTimes=np.nansum(train_counts) #用np计算NaN值的结果才是准确的
    #                 trainNonOxyToday.append([trainItem, trainTimes])
                    out['training']['detail'][trainTime]['NonOxy'][trainItem]=trainTimes
                else: #有氧
                    OxyTime=today_train[today_train['项目']==trainItem]['时长'].tolist()[0]
                    Kcal=today_train[today_train['项目']==trainItem]['消耗热量'].tolist()[0]
    #                 trainOxyToday.append([trainItem,OxyTime,Kcal])
                    out['training']['detail'][trainTime]['Oxy'][trainItem]=[OxyTime,Kcal]

        return json.dumps(out)
        
class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

    
    
        
if __name__=='__main__':
    my=Customs()
    p=my.read_xlsx('李测试')
    
    pp=json.loads(p)
    print(pp)
    
#     for k,v in pp['body'].items():
#         for i in v:
#             print(k,i)


In [203]:
import datetime


class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value
    
# d=Vividict()
# d['trainItem']['Oxy']['bike']=[30,400]
# d['trainItem']['NonOxy']['卷腹']=[0,40]
# d['trainItem']['NonOxy']['深蹲']=[0,60]
# print(d['trainItem']['NonOxy']['深蹲']
      

birth_d = datetime.datetime.strptime('19800523', "%Y%m%d")
today_d = datetime.datetime.now()
birth_t = birth_d.replace(year=2021)
      
print(birth_d,birth_t)

1980-05-23 00:00:00 2021-05-23 00:00:00
